# Regression Model A: Field 1

## Install libraries

In [1]:
install.packages('readxl')
install.packages('corrplot')
install.packages('RColorBrewer')
install.packages('car')
install.packages('Metrics')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("readxl"):
“installation of package ‘readxl’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


## Call libraries

In [2]:
library(readxl)
library(corrplot)
library(RColorBrewer)
library(car)
library(Metrics)

corrplot 0.84 loaded
Loading required package: carData


## Import data

In [3]:
d1=read_excel("variable2.xlsx",sheet = 1)
d2=read_excel("variable2.xlsx",sheet = 2)
d3=read_excel("variable2.xlsx",sheet = 3) 
d4=read_excel("variable2.xlsx",sheet = 4)
d5=read_excel("variable2.xlsx",sheet = 5) 

## Field 1 (F1)
Call the data according to the Field. 

In [4]:
dE1=d1[which(d1$`ESTATE CODE`=="E1" ),]
dE2=d2[(d2$Diff ),]
dE3=d3[which(d3$Estate=="E1" ),]
dE4=d4[which(d4$Estate=="E1" ),]
dE5=d5[which(d5$Estate=="E1" ),]

## Variables

Declare the variable and construct new table for easy representation. 

In [5]:
year=dE1$`CALENDER YEAR`
month=dE1$`CALENDER MONTH`
yield=dE1$`CPB TONNAGE`
rainfall=dE1$RAINFALL
foliar=d2$Diff
prun.time=dE3$Frequency
prun.cost=dE3$PruningCost
manuring.time=dE4$Frequency
manuring.cost=dE4$ManuringCost
pd.time=dE5$Frequency
pd.cost=dE5$PDCost

dg1=data.frame(year,month,yield,rainfall,foliar,prun.time,prun.cost,
               manuring.time,manuring.cost,pd.time,pd.cost)

## Correlation

Check the correlations for all variables. 

In [6]:
c1=cor(dg1[,-1:-2])

corrplot(c1, method="number",type="upper",col=brewer.pal(n=8, name="RdYlBu"))
corrplot(c1, type="upper")

## Regression

Fit the data using multiple regression for all variables. 

In [7]:
a1=lm(yield~rainfall+foliar+prun.time+prun.cost+
        manuring.time+manuring.cost+pd.time+pd.cost,data=dg1)
summary(a1)
plot(a1,which=1)
plot(a1,which=2)


Call:
lm(formula = yield ~ rainfall + foliar + prun.time + prun.cost + 
    manuring.time + manuring.cost + pd.time + pd.cost, data = dg1)

Residuals:
    Min      1Q  Median      3Q     Max 
-77.799 -37.511  -8.048  27.232 121.032 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    1.600e+02  3.636e+01   4.400 9.69e-05 ***
rainfall       1.509e-01  9.554e-02   1.579    0.123    
foliar        -1.440e+00  2.627e+00  -0.548    0.587    
prun.time     -1.827e+00  5.962e+00  -0.306    0.761    
prun.cost      4.698e-03  3.185e-03   1.475    0.149    
manuring.time -2.343e+01  1.797e+01  -1.304    0.201    
manuring.cost  2.281e-04  8.420e-04   0.271    0.788    
pd.time       -6.040e+00  1.055e+01  -0.573    0.571    
pd.cost       -3.829e-03  7.072e-03  -0.541    0.592    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 51.79 on 35 degrees of freedom
Multiple R-squared:  0.1937,	Adjusted R-squared:  0.00946

## Significant variable 

Check the significant variables using two ways. 

a.	anova  
Choose the variables that has significant value in which p<0.1.

b.	using stepwise method  
In order to avoid the multicollinearity issues in variables selection, we take into account the variance inflation factors (VIF). We remove the variables that has VIF greater than 4. Then we rebuild the model. 


The stepwise method incorporate with VIF factor is better than anova method in order to     select the significant variables.    

### Using ANOVA

In [8]:
anova(a1)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
rainfall,1,12182.88068,12182.88068,4.54141972,0.04018434
foliar,1,55.49196,55.49196,0.02068577,0.88646320
prun.time,1,373.54350,373.54350,0.13924603,0.71128230
prun.cost,1,3476.86306,3476.86306,1.29607233,0.26266476
manuring.time,1,2782.23361,2782.23361,1.03713489,0.31547710
manuring.cost,1,199.61268,199.61268,0.07440974,0.78662423
pd.time,1,2706.05102,2706.05102,1.00873626,0.32209933
pd.cost,1,786.14270,786.14270,0.29305088,0.59170147
Residuals,35,93891.52507,2682.61500,NA,NA


### Using stepwise

In [9]:
selectedMod <- step(a1)
summary(selectedMod)

##Avoid Multicollinearity and Statistical Significance
##Recursively remove variables with variance inflation factors: VIF > 4
# Remove vars with VIF> 4 and re-build model until none of VIFs don't exceed 4.
all_vifs <- car::vif(selectedMod)
print(all_vifs)

while(any(all_vifs > 4)){
  var_with_max_vif <- names(which(all_vifs == max(all_vifs)))  # get the var with max vif
  signif_all <- signif_all[!(signif_all) %in% var_with_max_vif]  # remove
  myForm <- as.formula(paste("ozone_reading ~ ", paste (signif_all, collapse=" + "), sep=""))  # new formula
  selectedMod <- lm(myForm, data=inputData)  # re-build model with new formula
  all_vifs <- car::vif(selectedMod)
}

summary(selectedMod)

Start:  AIC=355.29
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    manuring.cost + pd.time + pd.cost

                Df Sum of Sq    RSS    AIC
- manuring.cost  1     196.9  94088 353.38
- prun.time      1     251.8  94143 353.41
- pd.cost        1     786.1  94678 353.66
- foliar         1     806.4  94698 353.67
- pd.time        1     879.6  94771 353.70
<none>                        93892 355.29
- manuring.time  1    4560.8  98452 355.38
- prun.cost      1    5837.0  99729 355.94
- rainfall       1    6692.1 100584 356.32

Step:  AIC=353.38
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    pd.time + pd.cost

                Df Sum of Sq    RSS    AIC
- prun.time      1     410.6  94499 351.57
- pd.cost        1     685.3  94774 351.70
- pd.time        1    1032.6  95121 351.86
- foliar         1    1130.7  95219 351.91
<none>                        94088 353.38
- manuring.time  1    4421.1  98510 353.40
- prun.cost      1    5852.2  9


Call:
lm(formula = yield ~ rainfall, data = dg1)

Residuals:
    Min      1Q  Median      3Q     Max 
-76.957 -33.932  -1.002  27.616 113.793 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 123.66338   17.58804   7.031 1.31e-08 ***
rainfall      0.17404    0.07857   2.215   0.0322 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 49.83 on 42 degrees of freedom
Multiple R-squared:  0.1046,	Adjusted R-squared:  0.0833 
F-statistic: 4.907 on 1 and 42 DF,  p-value: 0.03222


ERROR: Error in vif.default(selectedMod): model contains fewer than 2 terms


## Prediction

Predict with the resample values.

In [10]:
set.seed(123)
dg3=sample(1:nrow(dg1),12)
dg4=dg1[dg3,]
estimate=predict(selectedMod,newdata=dg4)
estimate
sum.estimate=sum(estimate)
dg5=data.frame(estimate,sum.estimate)

13       34       18       37       38        2       21       43 
163.8667 137.4126 146.1146 170.8283 145.4184 145.9405 163.6927 174.4832 
      20       16       33       15 
164.5629 139.3270 187.7102 159.6897